In [1]:
import glob
import torch
import numpy as np
import torch.nn as nn
import scipy.io as sio
import matplotlib.pyplot as plt
import torch.nn.functional as F
import scipy.ndimage as ndimage
from torchvision import transforms, utils
from toolz.curried import pipe, curry, compose
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import matplotlib.pyplot as plt

import csv
import numpy as np


from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import os

import scores

import argparse
import datetime
import numpy as np
import math
import os
import gc

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader



## Reading in the Data

In [2]:
path_f=os.getcwd()

path_f_1=os.path.join(path_f, 'data')


names=[]
for files_txts in os.listdir(path_f_1):
    if files_txts.endswith(".csv"):
        #print(files_txts)
        names.append(files_txts)

print(names)
path_train=os.path.join(path_f_1, names[1])
path_test=os.path.join(path_f_1, names[0])

df_train=pd.read_csv(path_train)
print(df_train.shape)
print(path_train)

['test_data.csv', 'training_data.csv']
(2572, 99)
/gpfs/pace1/project/coc-fujimoto/dmo7/citrine_data_challenge/data/training_data.csv


## Feature Engineering

In [3]:
stab_vector=df_train['stabilityVec'].values
y=[]
for x in stab_vector:
    #print(x)
    a=np.fromstring(x[1:-1],sep=',').astype(int)
    y.append(a)
y=np.array(y) 

df_tmp = pd.DataFrame(y, columns = ['A', 'A91B', 'A82B','A73B','A64B','A55B','A46B','A37B','A28B','A19B','B'])
stab_vec_list=[ 'A91B', 'A82B','A73B','A64B','A55B','A46B','A37B','A28B','A19B']

df_train=df_train.drop("stabilityVec",axis=1) #removing the results which originally are a string
feature_cols=list(df_train)



df_train['formulaA']=df_train['formulaA_elements_Number']
df_train['formulaB']=df_train['formulaB_elements_Number']


df_train=pd.concat([df_train, df_tmp],axis=1)

y_all=df_train[stab_vec_list]
df_tmp_stable = pd.DataFrame( columns = ['Stable_compunds'])
df_tmp_stable['Stable_compunds']=np.logical_not(y_all.sum(axis=1)==0).astype(int) ## A one means it has a stable value  a 0 

df_train=pd.concat([df_train, df_tmp_stable],axis=1)
print(df_train.shape)


print(df_train.shape)


df_stable=df_train.loc[np.logical_not(y_all.sum(axis=1)==0)]
print(df_stable.shape)
df_stable.head()

(2572, 110)
(2572, 110)
(1228, 110)


,formulaA,formulaB,formulaA_elements_AtomicVolume,formulaB_elements_AtomicVolume,formulaA_elements_AtomicWeight,formulaB_elements_AtomicWeight,formulaA_elements_BoilingT,formulaB_elements_BoilingT,formulaA_elements_BulkModulus,formulaB_elements_BulkModulus,...,A82B,A73B,A64B,A55B,A46B,A37B,A28B,A19B,B,Stable_compunds
0,89,47,37.433086,17.075648,227.0,107.868200,3473.0,2435.0,0.0,100.0,...,0,1,0,1,0,0,0,0,1,1
1,89,13,37.433086,16.594425,227.0,26.981539,3473.0,2792.0,0.0,76.0,...,0,1,0,0,0,0,0,0,1,1
5,89,35,37.433086,42.527825,227.0,79.904000,3473.0,332.0,0.0,1.9,...,0,1,0,0,0,0,0,0,1,1
10,89,49,37.433086,26.082658,227.0,114.818000,3473.0,2345.0,0.0,0.0,...,0,1,0,0,0,0,1,0,1,1
28,89,81,37.433086,28.640877,227.0,204.383300,3473.0,1746.0,0.0,43.0,...,0,0,0,0,0,0,1,0,1,1


In [4]:
X_train_new=df_stable[feature_cols]
y_target=df_stable[stab_vec_list]

In [34]:
## Normalizing by Zscore
from scipy.stats import zscore
X_train_new_Z_score=X_train_new.apply(zscore)
print(X_train_new_Z_score.shape)

(1228, 98)


In [5]:
y_target.head()

,A91B,A82B,A73B,A64B,A55B,A46B,A37B,A28B,A19B
0,0,0,1,0,1,0,0,0,0
1,0,0,1,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0
10,0,0,1,0,0,0,0,1,0
28,0,0,0,0,0,0,0,1,0


## Neural network

In [26]:
import sys; sys.argv=['']; del sys
parser = argparse.ArgumentParser()

parser.add_argument_group('Optimization related arguments')
parser.add_argument('-num_epochs', default=200, type=int, help='Epochs')
parser.add_argument('-batch_size', default=64, type=int, help='Batch size')
parser.add_argument('-lr', default=1e-3, type=float, help='Learning rate')
parser.add_argument('-lr_decay_rate', default=0.9997592083, type=float, help='Decay for lr')
parser.add_argument('-min_lr', default=5e-5, type=float, help='Minimum learning rate')
parser.add_argument('-weight_init', default='xavier', choices=['xavier', 'kaiming'], help='Weight initialization strategy')
parser.add_argument('-overfit', action='store_true', help='Overfit on 5 examples, meant for debugging')
parser.add_argument('-gpuid', default=-1, type=int, help='GPU id to use')
        
parser.add_argument('-input_csv', default='./training_data.csv')
parser.add_argument('-normalize', default=True)
parser.add_argument('-test_size', default=0.33)

parser.add_argument_group('Checkpointing related arguments')
parser.add_argument('-load_path', default='', help='Checkpoint to load path from')
parser.add_argument('-save_path', default='checkpoints/', help='Path to save checkpoints')
parser.add_argument('-save_step', default=2, type=int, help='Save checkpoint after every save_step epochs')

# ----------------------------------------------------------------------------
# input arguments and options
# ----------------------------------------------------------------------------

args = parser.parse_args()

In [10]:
class arg_dataset(Dataset):
    """Face landmarks Dataset"""
    
    def __init__(self, data, y_true,test_size):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """        
        self.data=data
        self.y_true=y_true
        self.test_size = test_size
        assert len(data) == len(y_true)
        
    
        X_train, X_test, y_train, y_test = train_test_split(data, y_true,
                                                            test_size=test_size,
                                                            shuffle=True,
                                                            random_state=42)
        self.data = data

        self.X_train = torch.from_numpy(X_train.values).float()
        self.y_train = torch.from_numpy(y_train.values).float()
        self.X_test = torch.from_numpy(X_test.values).float()
        self.y_test = torch.from_numpy(y_test.values).float()

        self.num_data_points = {}
        self.num_data_points['train'] = len(X_train)
        self.num_data_points['test'] = len(X_test)
        
        self._split = 'train'

    @property
    def split(self):
        return self._split

    @split.setter
    def split(self, split):
        self._split = split

    # ------------------------------------------------------------------------
    # methods to override - __len__ and __getitem__ methods
    # ------------------------------------------------------------------------

    def __len__(self):
        return self.num_data_points[self._split]

    def __getitem__(self, idx):
        dtype = self._split
        item = {'index': idx}
        item['features'] = self.X_train[idx,:]
        item['outputs'] = self.y_train[idx,:]
        return item

    #-------------------------------------------------------------------------
    # collate function utilized by dataloader for batching
    #-------------------------------------------------------------------------

    def collate_fn(self, batch):
        dtype = self._split
        merged_batch = {key: [d[key] for d in batch] for key in batch[0]}
        out = {}
        for key in merged_batch:
            if key in {'index'}:
                out[key] = merged_batch[key]
            else:
                out[key] = torch.stack(merged_batch[key], 0)

        batch_keys = ['features', 'outputs']
        return {key: out[key] for key in batch_keys}



In [27]:
class Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.layers = nn.Sequential(
            nn.Linear(98, 60),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(60, 30),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(30, 15),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(15, 9)
        )
    def forward(self, batch):
        return self.layers(batch)

In [28]:
model_args = args

In [35]:
dataset = arg_dataset(X_train_new_Z_score, y_target,.33)
dataloader = DataLoader(dataset,
                        batch_size=args.batch_size,
                        shuffle=True,
                        collate_fn=dataset.collate_fn)

In [36]:
setattr(args, 'iter_per_epoch', math.ceil(dataset.num_data_points['train'] / args.batch_size))
print("{} iter per epoch.".format(args.iter_per_epoch))

13 iter per epoch.


In [37]:
net = Network(model_args)
optimizer = optim.Adam(net.parameters())
criterion = nn.MultiLabelSoftMarginLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=args.lr_decay_rate)

In [ ]:
# ----------------------------------------------------------------------------
# training
# ----------------------------------------------------------------------------

net.train()
os.makedirs(args.save_path, exist_ok=True)

running_loss = 0.0
train_begin = datetime.datetime.utcnow()
print("Training start time: {}".format(datetime.datetime.strftime(train_begin, '%d-%b-%Y-%H:%M:%S')))

log_loss = []
for epoch in range(1, model_args.num_epochs + 1):
    for i, batch in enumerate(dataloader):
        optimizer.zero_grad()
        for key in batch:
            batch[key] = Variable(batch[key])
            if args.gpuid >= 0:
                batch[key] = batch[key].cuda()

        # --------------------------------------------------------------------
        # forward-backward pass and optimizer step
        # --------------------------------------------------------------------
        net_out = net(batch['features'])

        cur_loss = criterion(net_out, batch['outputs'])
        cur_loss.backward()

        optimizer.step()
        gc.collect()

        # --------------------------------------------------------------------
        # update running loss and decay learning rates
        # --------------------------------------------------------------------
        #train_loss = cur_loss.data[0]
        train_loss = cur_loss.item()
        if running_loss > 0.0:
            running_loss = 0.95 * running_loss + 0.05 * train_loss
        else:
            running_loss = train_loss 

        if optimizer.param_groups[0]['lr'] > args.min_lr:
            scheduler.step()


        # --------------------------------------------------------------------
        # print after every few iterations
        # --------------------------------------------------------------------
        if i % 10 == 0:
            test_losses = []
            accuracy = []
            for i in range(int(dataset.num_data_points['test']/args.batch_size)):
                test_feat = dataset.X_test[i*args.batch_size:(i+1)*args.batch_size, :]
                test_labels = dataset.y_test[i*args.batch_size:(i+1)*args.batch_size, :]
                test_feat = Variable(test_feat)
                test_labels = Variable(test_labels)
                if args.gpuid >= 0:
                    test_feat = test_feat.cuda()
                    test_labels = test_labels.cuda()
                net_out = net(test_feat)
                cur_loss = criterion(net_out, test_labels)
                test_losses.append(cur_loss.item())
                
                y_pred = torch.sigmoid(net_out).data > 0.5
                y_pred = y_pred.cpu().numpy()
                accuracy.append((test_labels.cpu().numpy() == y_pred).all(axis=1))

            validation_loss = np.mean(test_losses)
            
            accuracy = np.mean(accuracy)

            iteration = (epoch - 1) * args.iter_per_epoch + i

            log_loss.append((epoch,
                             iteration,
                             running_loss,
                             train_loss,
                             validation_loss,
                             accuracy,
                             optimizer.param_groups[0]['lr']))

            # print current time, running average, learning rate, iteration, epoch
            print("[{}][Epoch: {:3d}][Iter: {:6d}][Loss: {:6f}][val loss: {:6f}][accuracy: {:6f}][lr: {:7f}]".format(
                datetime.datetime.utcnow() - train_begin, epoch,
                    iteration, running_loss, validation_loss, accuracy,
                    optimizer.param_groups[0]['lr']))

    # ------------------------------------------------------------------------
    # save checkpoints and final model
    # ------------------------------------------------------------------------
    if epoch % args.save_step == 0:
        torch.save({
            'net': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'model_args': net.args
        }, os.path.join(args.save_path, 'model_epoch_{}.pth'.format(epoch)))

torch.save({
    'net': net.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': net.args
}, os.path.join(args.save_path, 'model_final.pth'))

np.save(os.path.join(args.save_path, 'log_loss'), log_loss)

Training start time: 23-Jan-2019-05:32:05
[0:00:00.390055][Epoch:   1][Iter:      5][Loss: 0.669745][val loss: 0.666676][accuracy: 0.018229][lr: 0.001000]
[0:00:01.747813][Epoch:   1][Iter:      5][Loss: 0.664610][val loss: 0.646542][accuracy: 0.031250][lr: 0.000998]
[0:00:02.341572][Epoch:   2][Iter:     18][Loss: 0.661996][val loss: 0.638974][accuracy: 0.026042][lr: 0.000997]
[0:00:03.919283][Epoch:   2][Iter:     18][Loss: 0.646100][val loss: 0.599980][accuracy: 0.057292][lr: 0.000994]
[0:00:04.487911][Epoch:   3][Iter:     31][Loss: 0.638523][val loss: 0.587757][accuracy: 0.062500][lr: 0.000994]
[0:00:06.277722][Epoch:   3][Iter:     31][Loss: 0.606604][val loss: 0.524428][accuracy: 0.088542][lr: 0.000991]
[0:00:06.864728][Epoch:   4][Iter:     44][Loss: 0.594324][val loss: 0.509541][accuracy: 0.062500][lr: 0.000991]
[0:00:08.667184][Epoch:   4][Iter:     44][Loss: 0.554157][val loss: 0.477895][accuracy: 0.062500][lr: 0.000988]
[0:00:09.111633][Epoch:   5][Iter:     57][Loss: 0.543

[0:01:25.018104][Epoch:  37][Iter:    473][Loss: 0.385855][val loss: 0.391034][accuracy: 0.166667][lr: 0.000891]
[0:01:25.796281][Epoch:  38][Iter:    486][Loss: 0.386991][val loss: 0.392193][accuracy: 0.164062][lr: 0.000891]
[0:01:28.108865][Epoch:  38][Iter:    486][Loss: 0.385045][val loss: 0.390774][accuracy: 0.161458][lr: 0.000888]
[0:01:28.679656][Epoch:  39][Iter:    499][Loss: 0.380409][val loss: 0.387492][accuracy: 0.161458][lr: 0.000888]
[0:01:30.458209][Epoch:  39][Iter:    499][Loss: 0.377356][val loss: 0.384832][accuracy: 0.182292][lr: 0.000886]
[0:01:30.946014][Epoch:  40][Iter:    512][Loss: 0.381513][val loss: 0.388127][accuracy: 0.153646][lr: 0.000885]
[0:01:32.918407][Epoch:  40][Iter:    512][Loss: 0.380386][val loss: 0.390825][accuracy: 0.177083][lr: 0.000883]
[0:01:33.549798][Epoch:  41][Iter:    525][Loss: 0.379032][val loss: 0.390661][accuracy: 0.161458][lr: 0.000882]
[0:01:35.126626][Epoch:  41][Iter:    525][Loss: 0.378442][val loss: 0.386431][accuracy: 0.17708

[0:02:37.991931][Epoch:  74][Iter:    954][Loss: 0.354375][val loss: 0.376693][accuracy: 0.192708][lr: 0.000796]
[0:02:40.182070][Epoch:  74][Iter:    954][Loss: 0.350927][val loss: 0.377760][accuracy: 0.195312][lr: 0.000794]
[0:02:41.067468][Epoch:  75][Iter:    967][Loss: 0.352272][val loss: 0.378233][accuracy: 0.195312][lr: 0.000793]
[0:02:43.255662][Epoch:  75][Iter:    967][Loss: 0.350527][val loss: 0.377634][accuracy: 0.190104][lr: 0.000791]
[0:02:43.819618][Epoch:  76][Iter:    980][Loss: 0.350838][val loss: 0.381604][accuracy: 0.184896][lr: 0.000791]
[0:02:45.405721][Epoch:  76][Iter:    980][Loss: 0.349089][val loss: 0.381208][accuracy: 0.182292][lr: 0.000789]
[0:02:46.038227][Epoch:  77][Iter:    993][Loss: 0.347591][val loss: 0.382406][accuracy: 0.174479][lr: 0.000788]
[0:02:48.584743][Epoch:  77][Iter:    993][Loss: 0.351005][val loss: 0.383890][accuracy: 0.184896][lr: 0.000786]
[0:02:49.597480][Epoch:  78][Iter:   1006][Loss: 0.352526][val loss: 0.387877][accuracy: 0.18229

[0:04:08.149762][Epoch: 110][Iter:   1422][Loss: 0.330222][val loss: 0.381399][accuracy: 0.187500][lr: 0.000709]
[0:04:08.614528][Epoch: 111][Iter:   1435][Loss: 0.328895][val loss: 0.390295][accuracy: 0.221354][lr: 0.000709]
[0:04:10.096176][Epoch: 111][Iter:   1435][Loss: 0.329852][val loss: 0.393786][accuracy: 0.187500][lr: 0.000707]
[0:04:10.393427][Epoch: 112][Iter:   1448][Loss: 0.332715][val loss: 0.395040][accuracy: 0.187500][lr: 0.000706]
[0:04:11.407910][Epoch: 112][Iter:   1448][Loss: 0.331749][val loss: 0.385067][accuracy: 0.184896][lr: 0.000705]
[0:04:11.641832][Epoch: 113][Iter:   1461][Loss: 0.331810][val loss: 0.387956][accuracy: 0.190104][lr: 0.000704]
[0:04:12.994308][Epoch: 113][Iter:   1461][Loss: 0.330864][val loss: 0.387964][accuracy: 0.190104][lr: 0.000703]
[0:04:13.436163][Epoch: 114][Iter:   1474][Loss: 0.333095][val loss: 0.391090][accuracy: 0.203125][lr: 0.000702]
[0:04:15.080436][Epoch: 114][Iter:   1474][Loss: 0.328345][val loss: 0.395536][accuracy: 0.18750

In [24]:
y_pred[1]

array([0, 0, 1, 0, 1, 0, 0, 0, 0], dtype=uint8)

In [25]:
test_labels[1]

tensor([0., 0., 0., 0., 1., 0., 0., 0., 0.])